#### GMFlow Implementation

https://arxiv.org/abs/2111.13680

In [1]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from nnflow import GMFlow

2022-09-28 15:12:26.758662: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-28 15:12:27.086345: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-28 15:12:28.476871: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /shared/centos7/cuda/11.3/lib64:/shared/centos7/nodejs/14.15.4/lib
2022-09-28 15:12:28.477000: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.

In [3]:
from ezflow.models import build_model

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [12]:
def count_params(model):
    return str(sum(p.numel() for p in model.parameters() if p.requires_grad) / 1000000) + "M params"

##### Use `num_scales=1` and `upsample_factor=8` for GMFlow without refinement

In [26]:
model = build_model('GMFlow', cfg_path='../configs/gmflow/models/gmflow_v01.yaml', custom_cfg=True)

In [27]:
count_params(model) 

'4.680288M params'

In [6]:
# model = GMFlow(
#     feature_channels=128,
#     num_scales=1,
#     upsample_factor=8,
#     num_head=1,
#     attention_type='swin',
#     ffn_dim_expansion=4,
#     num_transformer_layers=6,
# )

In [23]:
img1, img2 = torch.randn(1,3,368,496), torch.randn(1,3,368,496)

In [28]:
# flow_result = model(img1, img2,
#        attn_splits_list=[2],
#        corr_radius_list=[-1],
#        prop_radius_list=[-1],
#    )

flow_result = model(img1, img2)

In [29]:
flow_result.keys()

dict_keys(['flow_preds'])

In [30]:
len(flow_result['flow_preds'])

2

In [31]:
for flow in flow_result['flow_preds']:
    print(flow.shape)

torch.Size([1, 2, 368, 496])
torch.Size([1, 2, 368, 496])


In [22]:
model.eval()

flow_result = model(img1, img2)
print(flow_result.keys())

flow_result['flow_preds'][0].shape, flow_result['flow_upsampled'].shape

dict_keys(['flow_preds', 'flow_upsampled'])


(torch.Size([1, 2, 256, 256]), torch.Size([1, 2, 256, 256]))

____

##### Use `num_scales=2` and `upsample_factor=8` for GMFlow with refinement

In [17]:
model_with_refinement = GMFlow(
    feature_channels=128,
    num_scales=2,
    upsample_factor=4,
    num_head=1,
    attention_type='swin',
    ffn_dim_expansion=4,
    num_transformer_layers=6,
)

In [19]:
flow_result = model_with_refinement(img1, img2,
       attn_splits_list=[2, 8],
       corr_radius_list=[-1, 4],
       prop_radius_list=[-1, 1],
   )

In [20]:
for flow in flow_result['flow_preds']:
    print(flow.shape)

torch.Size([1, 2, 256, 256])
torch.Size([1, 2, 256, 256])
torch.Size([1, 2, 256, 256])
torch.Size([1, 2, 256, 256])
